# Дополнительные данные (meteo.ru и ncep/ncar-реанализ по гидростанциям и региону)

In [176]:
import pandas as pd
import os
import glob
import re
import numpy as np
import functools
from functools import reduce
import math

In [2]:
path_to_data = '..//00_data/additional'

In [ ]:
# Метеостанции
meteo_stations = [24538, 24641, 24643, 24661, 24671, 24713, 24726, 24738, 24763,
       24817, 24923, 24933, 24944, 24951, 24959, 24966, 24967, 30028,
       30054, 30069, 30089, 30219, 30230, 30252, 30253, 30328, 30337,
       30356, 30372, 30385, 30393, 30433, 30471, 30493, 31004, 31026,
       31102, 31137]

# Дополнительные метеостанции
# 30230 - Киренск
# 30054 - Витим
# 24923 - Ленск
# 24944 - Олекминск
# 24856 - Покровск
# 24959 - Якутск
# 24652 - Сангар
# 24656 - Батамай

extra_meteo_stations = [30230, 30054, 24923, 24944, 24856, 24959, 24652, 24656]

meteo_stations = meteo_stations+extra_meteo_stations

In [ ]:
hydro_stations = [3019, 3021, 3024, 3027, 3028, 3029, 3030, 3031, 3032, 3035, 3036,
       3037, 3038, 3041, 3042, 3045, 3047, 3048, 3050, 3087, 3106, 3169,
       3180, 3229, 3230, 3554, 3555]

In [53]:
def reading_all_files(directory, list_of_stations):
    combined_file = pd.DataFrame()

    for file in list_of_stations:
        file = os.path.join(files, file)
        if os.path.isfile(file):
            temp = list(filter(None, [re.split('\s+', i.strip('\n')) for i in open(file)]))
            temp_pd = pd.DataFrame(temp)

            combined_file = combined_file.append(temp_pd)
    return combined_file

In [75]:
#stations_to_check = [24538]
#stations_to_check = map(str, stations_to_check)

In [59]:
#stations_to_check_snow = reading_all_files(files, stations_to_check)

## Meteo.ru

### Snow

In [ ]:
# List of all files to read in
#files = os.path.join(path_to_data, 'snow')
#file_list = glob.glob(files + '/*')

In [56]:
files = os.path.join(path_to_data, 'snow')

In [ ]:
# Create a dataframe for meteostations
file_list = map(str, meteo_stations)

meteo_snow = reading_all_files(files, file_list)

In [ ]:
#  KEY(U)        СТАНЦИЯ  FA(5)   PC(5);     // Индекс ВМО            
#  KEY(I)        ГОД      FA(5)   PC(4);     // Год                
#  KEY(I)        МЕСЯЦ    FA(3)   PC(2);     // Месяц              
#  KEY(I)        ДЕНЬ     FA(3)   PC(2);     // День        
#  MIT           СНЕПВЫСМ FC(5)   PC(4);     // Высота снежного покрова   
#  MIT           СНЕСТЕПВ FC(3)   PC(2);     // Степень покрытия окрестности станции снегом      
#  MIT           Q1       FC(2)   PC(1);     // Дополнительная информация о высоте снежного покрова 
#  MIT           Q2       FC(2)   PC(1);     // Признак качества по высоте снежного покрова 
#  MIT           Q3       FC(2)   PC(1);     // Дополнительная информация с учетом температуры воздуха 

#meteo_snow.columns = ['station_id', 'year', 'month', 'day', 'height_of_snow_cover',
#                        'how_much_snow_around', 'extra_info_about_snow_height',
#                         'some_property_of_snow_cover',
#                         'extra_info_about_snow_with_regard_to_air_temp'
#                        ]
meteo_snow.columns = ['station_id', 'Год', 'Месяц', 'День', 'Высота снежного покрова',
                        'Степень покрытия окрестности станции снегом', 'Дополнительная информация о высоте снежного покрова',
                         'Признак качества по высоте снежного покрова',
                         'Дополнительная информация с учетом температуры воздуха'
                        ]

In [ ]:
meteo_snow.head(2)

In [ ]:
meteo_snow.to_pickle(os.path.join(files, 'meteo_to_snow.pkl'))

In [ ]:
# Trying to create a dataframe for hydrostations - there is no info
file_list = map(str, hydro_stations)

hydro_snow = reading_all_files(files, file_list)

In [ ]:
hydro_snow.shape

### Tpg

In [ ]:
files = os.path.join(path_to_data, 'Tpg')

In [ ]:
file_list = map(str, meteo_stations)
file_list = ['st_' + s +'.dat' for s in file_list]

meteo_tpg = reading_all_files(files, file_list)

In [ ]:
meteo_tpg.head()

In [ ]:
# FAMILY TPG F(102); 
#   KEY(U)        СТАНЦИЯ  FA(5)   PC(5);         // Индекс ВМО            
#   KEY(I)        ГОД      FA(5)   PC(4);         // Год                
#   KEY(I)        МЕСЯЦ    FA(3)   PC(2);         // Месяц              
#   KEY(I)        ДЕНЬ     FA(3)   PC(2);         // День        
#   MIT           T02      FA(5)   PC(5,1) D(1);  // Температура на глубине 2 см                     
#   CHA(T02)      Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T05      FA(5)   PC(5,1) D(1);  // Температура на глубине 5 см                     
#   CHA(T05)      Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T10      FA(5)   PC(5,1) D(1);  // Температура на глубине 10 см                    
#  CHA(T10)      Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T15      FA(5)   PC(5,1) D(1);  // Температура на глубине 15 см                    
#   CHA(T15)      Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T20      FA(5)   PC(5,1) D(1);  // Температура на глубине 20 см                    
#   CHA(T20)      Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T40      FA(5)   PC(5,1) D(1);  // Температура на глубине 40 см                    
#   CHA(T40)      Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T60      FA(5)   PC(5,1) D(1);  // Температура на глубине 60 см                    
#   CHA(T60)      Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T80      FA(5)   PC(5,1) D(1);  // Температура на глубине 80 см                    
#   CHA(T80)      Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T120     FA(5)   PC(5,1) D(1);  // Температура на глубине 120 см                   
#   CHA(T120)     Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T160     FA(5)   PC(5,1) D(1);  // Температура на глубине 160 см                   
#   CHA(T160)     Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T240     FA(5)   PC(5,1) D(1);  // Температура на глубине 240 см                   
#   CHA(T240)     Q        FA(2)   PC(1) NA;      // Признак качества              
#   MIT           T320     FA(5)   PC(5,1) D(1);  // Температура на глубине 320 см                   
#   CHA(T320)     Q        FA(2)   PC(1) NA;      // Признак качества
meteo_tpg.columns = ['station_id', 'Год', 'Месяц', 'День', 'Температура на глубине 2 см ',
                     'Признак качества 1',
                        'Температура на глубине 5 см',   'Признак качества 2', 
                     'Температура на глубине 10 см',
                       'Признак качества 3',
                         'Температура на глубине 15 см',
                       'Признак качества 4',
                         'Температура на глубине 20 см',
                       'Признак качества 5',
                     'Температура на глубине 40 см', 
                       'Признак качества 6',
                     'Температура на глубине 60 см',
                       'Признак качества 7',
                         'Температура на глубине 80 см',
                       'Признак качества 8',
                         'Температура на глубине 120 см',
                       'Признак качества 9',
                     'Температура на глубине 160 см',
                       'Признак качества 10',
                         'Температура на глубине 240 см',
                       'Признак качества 11',
                         'Температура на глубине 320 см',
                      'Признак качества 12']

In [ ]:
meteo_tpg.to_pickle(os.path.join(files, 'meteo_to_tpg.pkl'))

### Tpgks

In [ ]:
files = os.path.join(path_to_data, 'Tpgks')

In [ ]:
file_list = map(str, meteo_stations)
file_list = ['st_' + s +'.dat' for s in file_list]

meteo_tpgks = reading_all_files(files, file_list)

In [ ]:
meteo_tpgks.head()

In [ ]:
# FAMILY TPGKS F(49);

#    KEY(U)        СТАНЦИЯ   FA(5) PC(5);        // Координатный номер станции                    
#    KEY(I)        ГОД       FA(5) PC(4);        // Год                               
#    KEY(I)        МЕСЯЦ     FA(3) PC(2);        // Месяц                             
#    KEY(I)        ДЕНЬ      FA(3) PC(2);        // День                             
#    MIT           ТЕМП05    FC(5) PC(4);  	// Температура на глубине  5 см, увеличенная в 10 раз
#    CHA(ТЕМП05)   Q         FA(3) PC(2) NA;     // Признак качества
#    MIT           ТЕМП10    FC(5) PC(4);  	// Температура на глубине 10 см, увеличенная в 10 раз
#    CHA(ТЕМП10)   Q         FA(3) PC(2) NA;     // Признак качества
#    MIT           ТЕМП15    FC(5) PC(4);  	// Температура на глубине 15 см, увеличенная в 10 раз
#    CHA(ТЕМП15)   Q         FA(3) PC(2) NA;     // Признак качества
#    MIT           ТЕМП20    FC(5) PC(4);  	// Температура на глубине 20 см, увеличенная в 10 раз
#    CHA(ТЕМП20)   Q         FA(3) PC(2) NA;    	// Признак качества


meteo_tpgks.columns = ['station_id', 'Год', 'Месяц', 'День', 'Температура на глубине  5 см, увеличенная в 10 раз',
                     'Признак качества 1',
                        'Температура на глубине 10 см, увеличенная в 10 раз',   'Признак качества 2', 
                     'Температура на глубине 15 см, увеличенная в 10 раз', 'Признак качества 3',
                         'Температура на глубине 20 см, увеличенная в 10 раз', 'Признак качества 4']

In [ ]:
meteo_tpgks.head()

In [ ]:
meteo_tpgks.shape

In [ ]:
meteo_tpgks.to_pickle(os.path.join(files, 'meteo_to_tpgks.pkl'))

### SnMar

In [ ]:
files = os.path.join(path_to_data, 'SnMar')

In [ ]:
file_list = map(str, meteo_stations)

meteo_SnMar = reading_all_files(files, file_list)

In [ ]:
#FAMILY SNMАR F(76); 

#  KEY(U)        СТАНЦИЯ  FA(5)   PC(5);        // Индекс ВМО            
#  KEY(I)        ГОД      FA(7)   PC(4);        // Год                
#  KEY(I)        МЕСЯЦ    FA(3)   PC(2);        // Месяц              
#  KEY(I)        ТИПМАРШ  FA(2)   PC(2);        // Тип маршрута              
#  KEY(I)        ДЕНЬ     FA(3)   PC(2);        // День        
#  MIT           СНЕСТЕПВ FC(3)   PC(2);        // Степень покрытия окрестности станции снегом      
#  MIT           СНЕСТПОК FC(3)   PC(2);        // Степень покрытия маршрута снегом 
#  MIT           СНЕСТПЛК FC(3)   PC(2);        // Степень покрытия маршрута ледяной коркой 
#  MIT           СНЕГСВЫС FC(5)   PC(4);        // Средняя высота снежного покрова на маршруте (см) 
#  MIT           СНЕВЫСМА FC(5)   PC(4);        // Наибольшая высота снежного покрова на маршруте (см)
#  MIT           СНЕВЫСМИ FC(5)   PC(4);        // Наименьшая высота снежного покрова на маршруте (см)
#  MIT           СНЕГСРПЛ FC(5,2) PC(4,2);      // Средняя плотность снега 
#  MIT           СНЕСРТЛК FC(4)   PC(3);        // Средняя толщина ледяной корки 
#  MIT           СНЕГТСНВ FC(4)   PC(3);        // Толщина слоя снега, насыщенного водой (мм) 
#  MIT           СНЕГТСЧВ FC(3)   PC(2);        // Толщина слоя чистой воды (мм) 
#  MIT           СПЗВСНЕГ FC(5)   PC(4);        // Запас воды в снеге (мм) 
#  MIT           СНПЗВОБЩ FC(5)   PC(4);        // Запас воды, общий (мм) 
#  MIT           СНЕГXАРЗ FC(2)   PC(1);        // Характер залегания снежного покрова 
#  MIT           СНЕГXАРП FC(2)   PC(1);        // Характер снежного покрова 

meteo_SnMar.columns = ['station_id', 'Год', 'Месяц', 'Тип маршрута', 'День', 
                       'Степень покрытия окрестности станции снегом',
                       'Степень покрытия маршрута снегом',
                       'Степень покрытия маршрута ледяной коркой',
                       'Средняя высота снежного покрова на маршруте (см)',
                       'Наибольшая высота снежного покрова на маршруте (см)',
                       'Наименьшая высота снежного покрова на маршруте (см)',
                       'Средняя плотность снега',
                       'Средняя толщина ледяной корки',
                       'Толщина слоя снега, насыщенного водой (мм)',
                       'Толщина слоя чистой воды (мм)',
                       'Запас воды в снеге (мм)',
                       'Запас воды, общий (мм)',
                       'Характер залегания снежного покрова',
                       'Характер снежного покрова']

In [ ]:
meteo_SnMar.to_pickle(os.path.join(files, 'meteo_to_SnMar.pkl'))

### TTTR

In [ ]:
files = os.path.join(path_to_data, 'TTTR')

In [ ]:
file_list = map(str, meteo_stations)

meteo_TTTR = reading_all_files(files, file_list)

In [ ]:
meteo_TTTR.head()

In [ ]:
#
#  KEY(U)        СТАНЦИЯ  FA(5)   PC(5);     // Индекс ВМО            
#  KEY(I)        ГОД      FA(5)   PC(4);     // Год                
#  KEY(I)        МЕСЯЦ    FA(3)   PC(2);     // Месяц              
#  KEY(I)        ДЕНЬ     FA(3)   PC(2);     // День        
#  MIT           QTTT     FA(2)   PC(1) NA;  // Общий признак качества температур                
#  MIT           ТЕВМИНСТ FC(6,1) PC(5,1);   // Минимальная температура воздуха  
#  CHA(ТЕВМИНСТ) Q        FA(2)   PC(1) NA;  // Признак качества                 
#  MIT           ТЕМВСРСТ FC(6,1) PC(5,1);   // Средняя температура воздуха      
#  CHA(ТЕМВСРСТ) Q        FA(2)   PC(1) NA;  // Признак качества                 
#  MIT           ТЕВМАКСТ FC(6,1) PC(5,1);   // Максимальная температура воздуха 
#  CHA(ТЕВМАКСТ) Q        FA(2)   PC(1) NA;  // Признак качества                 
#  MIT           ОСАДКОЛМ FC(6,1) PC(5,1);   // Количество осадков              
#  CHA(ОСАДКОЛМ) QО       FA(2)   PC(1) NA;  // Дополнительная характеристика   
#  CHA(ОСАДКОЛМ) Q        FA(2)   PC(1) NA;  // Признак качества
                                                                
meteo_TTTR.columns = ['station_id', 'Год', 'Месяц', 'День', 
                       'Общий признак качества температур',
                       ' Минимальная температура воздуха',
                       'Признак качества 1',
                       'Средняя температура воздуха',
                       'Признак качества 2',
                       'Максимальная температура воздуха',
                       'Признак качества 3',
                       'Количество осадков',
                      'Дополнительная характеристика',
                      'Признак качества 4']

In [ ]:
meteo_TTTR.to_pickle(os.path.join(files, 'meteo_to_TTTR.pkl'))

## Reanalysis

In [3]:
#!pip install netCDF4
#!pip install xarray
#!pip install ncdump

In [4]:
import netCDF4 as nc
import xarray as xr

### Reanalysis by station coordinates

In [ ]:
# Координаты метеостанаций
meteo_coord = pd.read_csv('..//00_data/track_1/meteo_coord.csv')
meteo_coord.head(2)
extra_meteo_coord = [[30230, 57.8,  108.1], [30054,  59.4,  112.6], 
                     [24923,  60.7,  114.9], [24944,  60.4,  120.4],
                     [24959,  62,  129.7], [24652,  64,  127.5],
                     [24656,  63.5,  129.5]]


extra_meteo_coord = pd.DataFrame(extra_meteo_coord, columns = ['station_id', 'lat', 'lon'])
meteo_coord = meteo_coord[['station_id', 'lat', 'lon']].append(extra_meteo_coord, ignore_index=True)

In [5]:
# Координаты гидростанций
hydro_stations_coord = [[3019,  57.77,  108.07], 
[3027,  59.45,  112.58333333333333],
[3028,  59.65,  112.8],
[3029,  59.73,  113.17],
[3030,  60.72,  114.88],
[3035,  60.36666666666667,  120.41666666666667],
[3041,  61.47,  129.17],
[3045,  62.01370819444445,  129.66808830555556],
[3050,  63.96,  127.41],
[3230,  63.516666666666666,  129.48333333333332]]
hydro_stations_coord = pd.DataFrame(hydro_stations_coord, columns = ['station_id', 'lat', 'lon'])

In [10]:
# For each station select reanalysis data according to the closest coordinate
def find_data_by_coordinates(list_of_files, list_of_stations):
    combined_dataframe = pd.DataFrame()
    for file in list_of_files:
        print(file)
        dataset = xr.open_dataset(file)
        for index, row in list_of_stations.iterrows():
            selected_rows = dataset.sel(lon = str(row['lon']), lat = str(row['lat']), method='nearest').to_dataframe()
            selected_rows['station_id'] = row['station_id']
            combined_dataframe = combined_dataframe.append(selected_rows)
    return combined_dataframe

#### Daily Air temperature

In [29]:
# What do we have in the air data?
test_file = xr.open_dataset(os.path.join(files, 'air.2021.nc'))
test_file

<xarray.Dataset>
Dimensions:  (lat: 73, level: 17, lon: 144, time: 584)
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 2021-01-01 ... 2021-05-26T18:00:00
Data variables:
    air      (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis (2014)
    history:        created 2013/12 by Hoop (netCDF2.3)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [79]:
files = os.path.join(path_to_data, 'ncer')
list_of_air_files = glob.glob(files + '/air*.nc')

In [15]:
air_data = find_data_by_coordinates(list_of_air_files, hydro_stations_coord)

..//00_data/additional\ncer\air.1985.nc
..//00_data/additional\ncer\air.1986.nc
..//00_data/additional\ncer\air.1987.nc
..//00_data/additional\ncer\air.1988.nc
..//00_data/additional\ncer\air.1989.nc
..//00_data/additional\ncer\air.1990.nc
..//00_data/additional\ncer\air.1991.nc
..//00_data/additional\ncer\air.1992.nc
..//00_data/additional\ncer\air.1993.nc
..//00_data/additional\ncer\air.1994.nc
..//00_data/additional\ncer\air.1995.nc
..//00_data/additional\ncer\air.1996.nc
..//00_data/additional\ncer\air.1997.nc
..//00_data/additional\ncer\air.1998.nc
..//00_data/additional\ncer\air.1999.nc
..//00_data/additional\ncer\air.2000.nc
..//00_data/additional\ncer\air.2001.nc
..//00_data/additional\ncer\air.2002.nc
..//00_data/additional\ncer\air.2003.nc
..//00_data/additional\ncer\air.2004.nc
..//00_data/additional\ncer\air.2005.nc
..//00_data/additional\ncer\air.2006.nc
..//00_data/additional\ncer\air.2007.nc
..//00_data/additional\ncer\air.2008.nc
..//00_data/additional\ncer\air.2009.nc


In [16]:
air_data.to_pickle(os.path.join(files, 'hydro_to_air.pkl'))

#### HGT (geopotential height data)

In [30]:
test_file = xr.open_dataset(os.path.join(files, 'hgt.1985.nc'))
test_file

<xarray.Dataset>
Dimensions:  (lat: 73, level: 17, lon: 144, time: 1460)
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1985-01-01 ... 1985-12-31T18:00:00
Data variables:
    hgt      (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC Reanalysis geopotential height data (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [17]:
list_of_hgt_files = glob.glob(files + '/hgt*.nc')

In [18]:
hgt_data = find_data_by_coordinates(list_of_hgt_files, hydro_stations_coord)

..//00_data/additional\ncer\hgt.1985.nc
..//00_data/additional\ncer\hgt.1986.nc
..//00_data/additional\ncer\hgt.1987.nc
..//00_data/additional\ncer\hgt.1988.nc
..//00_data/additional\ncer\hgt.1989.nc
..//00_data/additional\ncer\hgt.1990.nc
..//00_data/additional\ncer\hgt.1991.nc
..//00_data/additional\ncer\hgt.1992.nc
..//00_data/additional\ncer\hgt.1993.nc
..//00_data/additional\ncer\hgt.1994.nc
..//00_data/additional\ncer\hgt.1995.nc
..//00_data/additional\ncer\hgt.1996.nc
..//00_data/additional\ncer\hgt.1997.nc
..//00_data/additional\ncer\hgt.1998.nc
..//00_data/additional\ncer\hgt.1999.nc
..//00_data/additional\ncer\hgt.2000.nc
..//00_data/additional\ncer\hgt.2001.nc
..//00_data/additional\ncer\hgt.2002.nc
..//00_data/additional\ncer\hgt.2003.nc
..//00_data/additional\ncer\hgt.2004.nc
..//00_data/additional\ncer\hgt.2005.nc
..//00_data/additional\ncer\hgt.2006.nc
..//00_data/additional\ncer\hgt.2007.nc
..//00_data/additional\ncer\hgt.2008.nc
..//00_data/additional\ncer\hgt.2009.nc


In [19]:
hgt_data.to_pickle(os.path.join(files, 'hydro_to_hgt.pkl'))

In [26]:
hgt_data.head()

lat    lon    hgt  station_id
level  time                                               
1000.0 1985-01-01 00:00:00  57.5  107.5  226.0      3019.0
       1985-01-01 06:00:00  57.5  107.5  230.0      3019.0
       1985-01-01 12:00:00  57.5  107.5  246.0      3019.0
       1985-01-01 18:00:00  57.5  107.5  258.0      3019.0
       1985-01-02 00:00:00  57.5  107.5  278.0      3019.0

In [37]:
hgt_data.tail()

lat    lon      hgt  station_id
level time                                                 
10.0  2021-05-25 18:00:00  62.5  130.0  31462.0      3230.0
      2021-05-26 00:00:00  62.5  130.0  31448.0      3230.0
      2021-05-26 06:00:00  62.5  130.0  31550.0      3230.0
      2021-05-26 12:00:00  62.5  130.0  31514.0      3230.0
      2021-05-26 18:00:00  62.5  130.0  31480.0      3230.0

### Omega

In [31]:
test_file = xr.open_dataset(os.path.join(files, 'omega.1985.nc'))
test_file

<xarray.Dataset>
Dimensions:  (lat: 73, level: 12, lon: 144, time: 1460)
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 ... 250.0 200.0 150.0 100.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1985-01-01 ... 1985-12-31T18:00:00
Data variables:
    omega    (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis omega (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [20]:
list_of_omega_files = glob.glob(files + '/omega*.nc')

In [21]:
omega_data = find_data_by_coordinates(list_of_omega_files, hydro_stations_coord)

..//00_data/additional\ncer\omega.1985.nc
..//00_data/additional\ncer\omega.1986.nc
..//00_data/additional\ncer\omega.1987.nc
..//00_data/additional\ncer\omega.1988.nc
..//00_data/additional\ncer\omega.1989.nc
..//00_data/additional\ncer\omega.1990.nc
..//00_data/additional\ncer\omega.1991.nc
..//00_data/additional\ncer\omega.1992.nc
..//00_data/additional\ncer\omega.1993.nc
..//00_data/additional\ncer\omega.1994.nc
..//00_data/additional\ncer\omega.1995.nc
..//00_data/additional\ncer\omega.1996.nc
..//00_data/additional\ncer\omega.1997.nc
..//00_data/additional\ncer\omega.1998.nc
..//00_data/additional\ncer\omega.1999.nc
..//00_data/additional\ncer\omega.2000.nc
..//00_data/additional\ncer\omega.2001.nc
..//00_data/additional\ncer\omega.2002.nc
..//00_data/additional\ncer\omega.2003.nc
..//00_data/additional\ncer\omega.2004.nc
..//00_data/additional\ncer\omega.2005.nc
..//00_data/additional\ncer\omega.2006.nc
..//00_data/additional\ncer\omega.2007.nc
..//00_data/additional\ncer\omega.

In [22]:
omega_data.to_pickle(os.path.join(files, 'hydro_to_omega.pkl'))

### RHUM (rel. humidity)

In [81]:
test_file = xr.open_dataset(os.path.join(files, 'rhum.1985.nc'))
test_file

<xarray.Dataset>
Dimensions:  (lat: 73, level: 8, lon: 144, time: 1460)
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 600.0 500.0 400.0 300.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1985-01-01 ... 1985-12-31T18:00:00
Data variables:
    rhum     (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis rel. humidity (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [23]:
list_of_rhum_files = glob.glob(files + '/rhum*.nc')

In [24]:
rhum_data = find_data_by_coordinates(list_of_rhum_files, hydro_stations_coord)

..//00_data/additional\ncer\rhum.1985.nc
..//00_data/additional\ncer\rhum.1986.nc
..//00_data/additional\ncer\rhum.1987.nc
..//00_data/additional\ncer\rhum.1988.nc
..//00_data/additional\ncer\rhum.1989.nc
..//00_data/additional\ncer\rhum.1990.nc
..//00_data/additional\ncer\rhum.1991.nc
..//00_data/additional\ncer\rhum.1992.nc
..//00_data/additional\ncer\rhum.1993.nc
..//00_data/additional\ncer\rhum.1994.nc
..//00_data/additional\ncer\rhum.1995.nc
..//00_data/additional\ncer\rhum.1996.nc
..//00_data/additional\ncer\rhum.1997.nc
..//00_data/additional\ncer\rhum.1998.nc
..//00_data/additional\ncer\rhum.1999.nc
..//00_data/additional\ncer\rhum.2000.nc
..//00_data/additional\ncer\rhum.2001.nc
..//00_data/additional\ncer\rhum.2002.nc
..//00_data/additional\ncer\rhum.2003.nc
..//00_data/additional\ncer\rhum.2004.nc
..//00_data/additional\ncer\rhum.2005.nc
..//00_data/additional\ncer\rhum.2006.nc
..//00_data/additional\ncer\rhum.2007.nc
..//00_data/additional\ncer\rhum.2008.nc
..//00_data/addi

In [25]:
rhum_data.to_pickle(os.path.join(files, 'hydro_to_rhum.pkl'))

#### SHUM (spec. humidity)

In [34]:
test_file = xr.open_dataset(os.path.join(files, 'shum.1985.nc'))
test_file

<xarray.Dataset>
Dimensions:  (lat: 73, level: 8, lon: 144, time: 1460)
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 600.0 500.0 400.0 300.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1985-01-01 ... 1985-12-31T18:00:00
Data variables:
    shum     (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis spec. humidity (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/06/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [38]:
list_of_shum_files = glob.glob(files + '/shum*.nc')

In [40]:
shum_data = find_data_by_coordinates(list_of_shum_files, hydro_stations_coord)

..//00_data/additional\ncer\shum.1985.nc
..//00_data/additional\ncer\shum.1986.nc
..//00_data/additional\ncer\shum.1987.nc
..//00_data/additional\ncer\shum.1988.nc
..//00_data/additional\ncer\shum.1989.nc
..//00_data/additional\ncer\shum.1990.nc
..//00_data/additional\ncer\shum.1992.nc
..//00_data/additional\ncer\shum.1993.nc
..//00_data/additional\ncer\shum.1994.nc
..//00_data/additional\ncer\shum.1995.nc
..//00_data/additional\ncer\shum.1996.nc
..//00_data/additional\ncer\shum.1997.nc
..//00_data/additional\ncer\shum.1998.nc
..//00_data/additional\ncer\shum.1999.nc
..//00_data/additional\ncer\shum.2000.nc
..//00_data/additional\ncer\shum.2001.nc
..//00_data/additional\ncer\shum.2002.nc
..//00_data/additional\ncer\shum.2003.nc
..//00_data/additional\ncer\shum.2004.nc
..//00_data/additional\ncer\shum.2005.nc
..//00_data/additional\ncer\shum.2006.nc
..//00_data/additional\ncer\shum.2007.nc
..//00_data/additional\ncer\shum.2008.nc
..//00_data/additional\ncer\shum.2009.nc
..//00_data/addi

In [42]:
shum_data.to_pickle(os.path.join(files, 'hydro_to_shum.pkl'))

#### VWND

In [120]:
# A positive v wind is from the south, and a negative v wind is from the north.

In [82]:
test_file = xr.open_dataset(os.path.join(files, 'vwnd.1985.nc'))
test_file

<xarray.Dataset>
Dimensions:  (lat: 73, level: 17, lon: 144, time: 1460)
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1985-01-01 ... 1985-12-31T18:00:00
Data variables:
    vwnd     (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis v wind (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [110]:
list_of_vwnd_files = glob.glob(files + '/vwnd*.nc')

In [85]:
vwnd_data = find_data_by_coordinates(list_of_vwnd_files, hydro_stations_coord)

..//00_data/additional\ncer\vwnd.1985.nc
..//00_data/additional\ncer\vwnd.1986.nc
..//00_data/additional\ncer\vwnd.1987.nc
..//00_data/additional\ncer\vwnd.1988.nc
..//00_data/additional\ncer\vwnd.1989.nc
..//00_data/additional\ncer\vwnd.1990.nc
..//00_data/additional\ncer\vwnd.1991.nc
..//00_data/additional\ncer\vwnd.1992.nc
..//00_data/additional\ncer\vwnd.1993.nc
..//00_data/additional\ncer\vwnd.1994.nc
..//00_data/additional\ncer\vwnd.1995.nc
..//00_data/additional\ncer\vwnd.1996.nc
..//00_data/additional\ncer\vwnd.1997.nc
..//00_data/additional\ncer\vwnd.1998.nc
..//00_data/additional\ncer\vwnd.1999.nc
..//00_data/additional\ncer\vwnd.2000.nc
..//00_data/additional\ncer\vwnd.2001.nc
..//00_data/additional\ncer\vwnd.2002.nc
..//00_data/additional\ncer\vwnd.2003.nc
..//00_data/additional\ncer\vwnd.2004.nc
..//00_data/additional\ncer\vwnd.2005.nc
..//00_data/additional\ncer\vwnd.2006.nc
..//00_data/additional\ncer\vwnd.2007.nc
..//00_data/additional\ncer\vwnd.2008.nc
..//00_data/addi

In [86]:
vwnd_data.to_pickle(os.path.join(files, 'hydro_to_vwnd.pkl'))

#### UWND

In [119]:
# A positive u wind is from the west. A negative u wind is from the east.

In [88]:
test_file = xr.open_dataset(os.path.join(files, 'uwnd.1985.nc'))
test_file

<xarray.Dataset>
Dimensions:  (lat: 73, level: 17, lon: 144, time: 1460)
Coordinates:
  * level    (level) float32 1e+03 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1985-01-01 ... 1985-12-31T18:00:00
Data variables:
    uwnd     (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis u wind (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/02/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [109]:
list_of_uwnd_files = glob.glob(files + '/uwnd*.nc')

In [97]:
uwnd_data = find_data_by_coordinates(list_of_uwnd_files, hydro_stations_coord)

..//00_data/additional\ncer\uwnd.1985.nc
..//00_data/additional\ncer\uwnd.1986.nc
..//00_data/additional\ncer\uwnd.1987.nc
..//00_data/additional\ncer\uwnd.1988.nc
..//00_data/additional\ncer\uwnd.1989.nc
..//00_data/additional\ncer\uwnd.1990.nc
..//00_data/additional\ncer\uwnd.1991.nc
..//00_data/additional\ncer\uwnd.1992.nc
..//00_data/additional\ncer\uwnd.1993.nc
..//00_data/additional\ncer\uwnd.1994.nc
..//00_data/additional\ncer\uwnd.1995.nc
..//00_data/additional\ncer\uwnd.1996.nc
..//00_data/additional\ncer\uwnd.1997.nc
..//00_data/additional\ncer\uwnd.1998.nc
..//00_data/additional\ncer\uwnd.1999.nc
..//00_data/additional\ncer\uwnd.2000.nc
..//00_data/additional\ncer\uwnd.2001.nc
..//00_data/additional\ncer\uwnd.2002.nc
..//00_data/additional\ncer\uwnd.2003.nc
..//00_data/additional\ncer\uwnd.2004.nc
..//00_data/additional\ncer\uwnd.2005.nc
..//00_data/additional\ncer\uwnd.2006.nc
..//00_data/additional\ncer\uwnd.2007.nc
..//00_data/additional\ncer\uwnd.2008.nc
..//00_data/addi

In [98]:
uwnd_data.to_pickle(os.path.join(files, 'hydro_to_uwnd.pkl'))

### Join the selected NCEP/NCAR data

In [44]:
shum_data.head(2)

lat    lon     shum  station_id
level  time                                                 
1000.0 1985-01-01 00:00:00  57.5  107.5  0.00098      3019.0
       1985-01-01 06:00:00  57.5  107.5  0.00096      3019.0

In [130]:
# Take level 1000 and join by station Id and time
air_data_1000 = air_data[air_data.index.isin([1000], level = 0)]
hgt_data_1000 = hgt_data[hgt_data.index.isin([1000], level = 0)]
uwnd_data_1000 = uwnd_data[uwnd_data.index.isin([1000], level = 0)]
vwnd_data_1000 = vwnd_data[vwnd_data.index.isin([1000], level = 0)]
shum_data_1000 = shum_data[shum_data.index.isin([1000], level = 0)]
rhum_data_1000 = rhum_data[rhum_data.index.isin([1000], level = 0)]
omega_data_1000 = omega_data[omega_data.index.isin([1000], level = 0)]

In [95]:
hgt_data_1000.tail()

lat    lon   hgt  station_id
level  time                                              
1000.0 2021-05-25 18:00:00  62.5  130.0  69.0      3230.0
       2021-05-26 00:00:00  62.5  130.0  57.0      3230.0
       2021-05-26 06:00:00  62.5  130.0  27.0      3230.0
       2021-05-26 12:00:00  62.5  130.0  10.0      3230.0
       2021-05-26 18:00:00  62.5  130.0  18.0      3230.0

In [131]:
all_data_1000 = [air_data_1000, hgt_data_1000, uwnd_data_1000, vwnd_data_1000, shum_data_1000, rhum_data_1000, omega_data_1000]
all_data_1000_df = reduce(lambda left, right: pd.merge(left, right, on=['time', 'station_id', 'lat', 'lon']), all_data_1000)

In [133]:
all_data_1000_df.head()

,lat,lon,air,station_id,hgt,uwnd,vwnd,shum,rhum,omega
time,,,,,,,,,,
1985-01-01 00:00:00,57.5,107.5,256.500000,3019.0,226.0,0.100006,1.600006,0.00098,94.000015,0.054998
1985-01-01 06:00:00,57.5,107.5,255.400009,3019.0,230.0,1.400009,3.500015,0.00096,100.000015,0.066998
1985-01-01 12:00:00,57.5,107.5,257.000000,3019.0,246.0,-1.099991,3.400009,0.00109,100.000015,0.036999
1985-01-01 18:00:00,57.5,107.5,258.799988,3019.0,258.0,-0.199997,2.300003,0.00121,96.000015,0.007998
1985-01-02 00:00:00,57.5,107.5,256.700012,3019.0,278.0,2.500015,1.200012,0.00092,86.000015,0.033998


In [107]:
all_data_1000_df.tail()

,lat,lon,hgt,station_id,uwnd,vwnd,shum,rhum,omega
time,,,,,,,,,
2021-05-25 18:00:00,62.5,130.0,69.0,3230.0,-4.499999,0.000002,0.00498,99.0,0.041
2021-05-26 00:00:00,62.5,130.0,57.0,3230.0,-4.399999,-1.300000,0.00523,66.0,-0.081
2021-05-26 06:00:00,62.5,130.0,27.0,3230.0,-2.899997,0.100002,0.00725,76.0,0.042
2021-05-26 12:00:00,62.5,130.0,10.0,3230.0,-1.199998,0.200002,0.00748,73.0,0.033
2021-05-26 18:00:00,62.5,130.0,18.0,3230.0,-1.499998,-0.199997,0.00586,94.0,0.055


In [106]:
all_data_1000_df.shape

(517200, 9)

In [132]:
all_data_1000_df.to_pickle(os.path.join(path_to_data, 'all_pressure_data_level_1000.pkl'))

## NCEP/NCAR data by region

In [124]:
# Considered region (level == 1000)
#lat: Begin: 
#57.00N
# End: 
# 64.00N

#lon: Begin: 
#107E
# End: 
#136.00E

### U-Wind 

In [163]:
# Read in selected data
# (https://psl.noaa.gov/cgi-bin/DataAccess.pl?DB_dataset=NCEP+Reanalysis+Daily+Averages&DB_variable=u-wind&DB_statistic=Mean&DB_tid=92831&DB_did=198&DB_vid=666)
#file = (os.path.join(path_to_data, 'ncer/region/X93.231.116.189.148.5.53.20.nc'))
file = (os.path.join(path_to_data, 'ncer/region/X93.231.116.189.148.6.29.35.nc'))

dataset_uwnd = xr.open_dataset(file)

In [141]:
dataset

<xarray.Dataset>
Dimensions:  (lat: 3, level: 1, lon: 12, time: 13270)
Coordinates:
  * level    (level) float32 1e+03
  * lat      (lat) float32 62.5 60.0 57.5
  * lon      (lon) float32 107.5 110.0 112.5 115.0 ... 127.5 130.0 132.5 135.0
  * time     (time) datetime64[ns] 1985-01-01 1985-01-02 ... 2021-05-01
Data variables:
    uwnd     (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis u wind (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        Sat May 29 06:29:36 2021: ncrcat -O -d lon,107.000000,136...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...
    NCO:            netCDF Operators version 4.8.1 (Homepage = http://nco.sf....

In [165]:
dataset_uwnd = dataset_uwnd.to_dataframe().reset_index()
dataset_uwnd['uwnd'] = dataset_uwnd['uwnd'].astype(np.float)
dataset_uwnd.head()

C:\Users\Tanya\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,lat,level,lon,time,uwnd
0,62.5,1000.0,107.5,1985-01-01,1.230011
1,62.5,1000.0,107.5,1985-01-02,-1.389999
2,62.5,1000.0,107.5,1985-01-03,0.550003
3,62.5,1000.0,107.5,1985-01-04,3.550003
4,62.5,1000.0,107.5,1985-01-05,2.360001


### V-Wind

In [166]:
# Read in selected data
# https://psl.noaa.gov/cgi-bin/GrADS.pl?dataset=NCEP+Reanalysis+Daily+Averages&DB_did=198&file=%2FDatasets%2Fncep.reanalysis.dailyavgs%2Fpressure%2Fvwnd.1948.nc+vwnd.%25y4.nc+26810&variable=vwnd&DB_vid=667&DB_tid=92831&units=m%2Fs&longstat=Mean&DB_statistic=Mean&stat=&lat-begin=57.00N&lat-end=64.00N&lon-begin=107.00E&lon-end=136.00E&dim0=time&year_begin=1985&mon_begin=Jan&day_begin=1&year_end=2021&mon_end=May&day_end=1&dim1=level&level+units=millibar&level=1000&X=lon&Y=lat&output=file&bckgrnd=black&use_color=on&fill=lines&cint=&range1=&range2=&scale=100&maskf=%2FDatasets%2Fncep.reanalysis.dailyavgs%2Fsurface%2Fland.nc&maskv=Land-sea+mask&submit=Create+Plot+or+Subset+of+Data
file = (os.path.join(path_to_data, 'ncer/region/X93.231.116.189.148.13.52.2.nc'))

dataset_vwnd = xr.open_dataset(file)

In [167]:
dataset_vwnd

<xarray.Dataset>
Dimensions:  (lat: 3, level: 1, lon: 12, time: 13270)
Coordinates:
  * level    (level) float32 1e+03
  * lat      (lat) float32 62.5 60.0 57.5
  * lon      (lon) float32 107.5 110.0 112.5 115.0 ... 127.5 130.0 132.5 135.0
  * time     (time) datetime64[ns] 1985-01-01 1985-01-02 ... 2021-05-01
Data variables:
    vwnd     (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis v wind (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        Sat May 29 13:52:02 2021: ncrcat -O -d lon,107.000000,136...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...
    NCO:            netCDF Operators version 4.8.1 (Homepage = http://nco.sf....

In [168]:
dataset_vwnd = dataset_vwnd.to_dataframe().reset_index()
dataset_vwnd['vwnd'] = dataset_vwnd['vwnd'].astype(np.float)
dataset_vwnd.head()

C:\Users\Tanya\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,lat,level,lon,time,vwnd
0,62.5,1000.0,107.5,1985-01-01,3.180008
1,62.5,1000.0,107.5,1985-01-02,1.230011
2,62.5,1000.0,107.5,1985-01-03,0.360001
3,62.5,1000.0,107.5,1985-01-04,0.080002
4,62.5,1000.0,107.5,1985-01-05,0.860001


In [173]:
dataframe_wind = pd.merge(dataset_vwnd, dataset_uwnd, on=['time','lon', 'lat'])

In [174]:
dataframe_wind

,lat,level_x,lon,time,vwnd,level_y,uwnd
0,62.5,1000.0,107.5,1985-01-01,3.180008,1000.0,1.230011
1,62.5,1000.0,107.5,1985-01-02,1.230011,1000.0,-1.389999
2,62.5,1000.0,107.5,1985-01-03,0.360001,1000.0,0.550003
3,62.5,1000.0,107.5,1985-01-04,0.080002,1000.0,3.550003
4,62.5,1000.0,107.5,1985-01-05,0.860001,1000.0,2.360001
...,...,...,...,...,...,...,...
477715,57.5,1000.0,135.0,2021-04-27,3.075002,1000.0,-4.324998
477716,57.5,1000.0,135.0,2021-04-28,0.825000,1000.0,-3.674999
477717,57.5,1000.0,135.0,2021-04-29,-0.974999,1000.0,-3.199999
477718,57.5,1000.0,135.0,2021-04-30,-0.674998,1000.0,-3.074999


In [183]:
# Wind speed 
# ws = sqrt(u2+v2)
dataframe_wind['wind_speed'] = np.sqrt((dataframe_wind['vwnd']*dataframe_wind['vwnd']) + (dataframe_wind['uwnd']*dataframe_wind['uwnd']))

In [184]:
dataframe_wind.head()

,lat,level_x,lon,time,vwnd,level_y,uwnd,wind_speed
0,62.5,1000.0,107.5,1985-01-01,3.180008,1000.0,1.230011,3.409601
1,62.5,1000.0,107.5,1985-01-02,1.230011,1000.0,-1.389999,1.856078
2,62.5,1000.0,107.5,1985-01-03,0.360001,1000.0,0.550003,0.657346
3,62.5,1000.0,107.5,1985-01-04,0.080002,1000.0,3.550003,3.550904
4,62.5,1000.0,107.5,1985-01-05,0.860001,1000.0,2.360001,2.511813


## Air temperature

In [186]:
# Read in selected data
# https://psl.noaa.gov/cgi-bin/GrADS.pl?dataset=NCEP+Reanalysis+Daily+Averages&DB_did=198&file=%2FDatasets%2Fncep.reanalysis.dailyavgs%2Fpressure%2Fair.1948.nc+air.%25y4.nc+26810&variable=air&DB_vid=4984&DB_tid=92831&units=degK&longstat=Mean&DB_statistic=Mean&stat=&lat-begin=57.00N&lat-end=64.00N&lon-begin=107E&lon-end=136.00E&dim0=level&level+units=millibar&level=1000&dim1=time&year_begin=1985&mon_begin=Jan&day_begin=1&year_end=2021&mon_end=May&day_end=1&X=lon&Y=lat&output=file&bckgrnd=black&use_color=on&fill=lines&cint=&range1=&range2=&scale=100&maskf=%2FDatasets%2Fncep.reanalysis.dailyavgs%2Fsurface%2Fland.nc&maskv=Land-sea+mask&submit=Create+Plot+or+Subset+of+Dat
file = (os.path.join(path_to_data, 'ncer/region/X93.231.116.189.148.14.4.7.nc'))

dataset_air = xr.open_dataset(file)

In [187]:
dataset_air

<xarray.Dataset>
Dimensions:  (lat: 3, level: 1, lon: 12, time: 13270)
Coordinates:
  * level    (level) float32 1e+03
  * lat      (lat) float32 62.5 60.0 57.5
  * lon      (lon) float32 107.5 110.0 112.5 115.0 ... 127.5 130.0 132.5 135.0
  * time     (time) datetime64[ns] 1985-01-01 1985-01-02 ... 2021-05-01
Data variables:
    air      (time, level, lat, lon) float32 ...
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis temperature (1985)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        Sat May 29 14:04:08 2021: ncrcat -O -d lon,107.000000,136...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...
    NCO:            netCDF Operators version 4.8.1 (Homepage = http://nco.sf....

In [189]:
dataset_air = dataset_air.to_dataframe().reset_index()
dataset_air['air'] = dataset_air['air'].astype(np.float)
dataset_air.head()

C:\Users\Tanya\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,lat,level,lon,time,air
0,62.5,1000.0,107.5,1985-01-01,245.730011
1,62.5,1000.0,107.5,1985-01-02,246.730011
2,62.5,1000.0,107.5,1985-01-03,246.260010
3,62.5,1000.0,107.5,1985-01-04,244.730011
4,62.5,1000.0,107.5,1985-01-05,244.930008


In [190]:
dataframe_wind_air = pd.merge(dataset_air, dataframe_wind, on=['time','lon', 'lat'])

In [191]:
dataframe_wind_air.head(2)

,lat,level,lon,time,air,level_x,vwnd,level_y,uwnd,wind_speed
0,62.5,1000.0,107.5,1985-01-01,245.730011,1000.0,3.180008,1000.0,1.230011,3.409601
1,62.5,1000.0,107.5,1985-01-02,246.730011,1000.0,1.230011,1000.0,-1.389999,1.856078


## Calculating overall region statistics

In [192]:
statistics_per_day = dataframe_wind_air.groupby('time').agg({
    'uwnd': [min, max, sum, np.mean],
    'vwnd': [min, max, sum, np.mean],
    'air': [min, max, sum, np.mean],
    'wind_speed': [min, max, sum, np.mean],
})

In [194]:
statistics_per_day.columns = [' '.join(col).strip() for col in statistics_per_day.columns.values]

In [197]:
statistics_per_day.head()

,uwnd min,uwnd max,uwnd sum,uwnd mean,vwnd min,vwnd max,vwnd sum,vwnd mean,air min,air max,air sum,air mean,wind_speed min,wind_speed max,wind_speed sum,wind_speed mean
time,,,,,,,,,,,,,,,,
1985-01-01,-1.039993,4.630005,71.640244,1.990007,-3.689987,3.750015,32.140274,0.892785,242.100006,257.080017,8929.160278,248.032230,0.872345,5.920553,116.218287,3.228286
1985-01-02,-1.389999,6.930008,112.470276,3.124174,-7.339996,3.560013,-12.669693,-0.351936,241.310013,255.550003,8902.270294,247.285286,0.639536,9.220193,159.361553,4.426710
1985-01-03,0.500015,6.960007,128.440216,3.567784,-7.019989,4.680008,-26.459747,-0.734993,243.030014,256.350006,8926.570282,247.960286,0.657346,8.481624,182.329743,5.064715
1985-01-04,-0.519989,4.750015,124.590286,3.460841,-5.369995,1.280014,-47.799683,-1.327769,240.830002,251.880005,8877.980270,246.610563,1.136005,6.696041,149.619400,4.156094
1985-01-05,-1.289993,6.560013,141.080276,3.918897,-6.369995,3.460007,-5.099655,-0.141657,239.600006,249.130005,8832.920319,245.358898,1.308178,8.668985,178.633276,4.962035


In [196]:
statistics_per_day.to_pickle(os.path.join(files, 'statistics_per_day.pkl'))

## Aggregate region statistics per week/month/season

In [199]:
statistics_per_week = statistics_per_day.resample('W').agg({
    'uwnd min':min, 
    'uwnd min': min,
    'uwnd mean': np.mean, # mean of means :)
    'vwnd sum':  np.sum, 
    'vwnd sum':  np.sum, 
    'vwnd mean': np.mean, 
    'air min': min,
    'air max': max,
    'air sum': np.sum,
    'air mean': np.mean, 
    'wind_speed min': min,
    'wind_speed max':max,
    'wind_speed sum':np.sum,
    'wind_speed mean': np.mean
})

In [200]:
statistics_per_week

,uwnd min,uwnd mean,vwnd sum,vwnd mean,air min,air max,air sum,air mean,wind_speed min,wind_speed max,wind_speed sum,wind_speed mean
time,,,,,,,,,,,,
1985-01-06,-1.389999,3.144451,4.911743,0.022740,238.980011,257.080017,53374.281738,247.103156,0.639536,9.220193,937.533959,4.340435
1985-01-13,-2.219986,1.526118,121.872025,0.483619,229.550003,258.049988,60888.421921,241.620722,0.313839,6.007800,726.899264,2.884521
1985-01-20,-3.439987,1.477746,320.021912,1.269928,228.200012,260.780029,61351.092117,243.456715,0.270189,7.934622,818.358010,3.247452
1985-01-27,-4.599991,1.223817,55.281891,0.219373,231.880005,253.480011,61097.141846,242.448976,0.072790,6.937333,676.136211,2.683080
1985-02-03,-4.169998,2.088936,146.611877,0.581793,236.510010,264.799988,62453.222015,247.830246,0.244128,7.882705,870.184420,3.453113
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-04,-4.100000,1.333532,114.125233,0.452878,253.124985,274.850006,67096.121826,266.254452,0.200000,9.864393,774.514505,3.073470
2021-04-11,-5.224999,1.606251,-51.099695,-0.202777,252.399994,279.699982,67061.797226,266.118243,0.509904,7.614173,982.767435,3.899871
2021-04-18,-3.624999,2.293156,488.175289,1.937204,261.250000,278.724976,67950.121460,269.643339,0.760345,11.086085,1078.075336,4.278077


In [201]:
statistics_per_week.to_pickle(os.path.join(files, 'statistics_per_week.pkl'))

In [203]:
statistics_per_month = statistics_per_day.resample('M').agg({
    'uwnd min':min, 
    'uwnd min': min,
    'uwnd mean': np.mean, # mean of means :)
    'vwnd sum':  np.sum, 
    'vwnd sum':  np.sum, 
    'vwnd mean': np.mean, 
    'air min': min,
    'air max': max,
    'air sum': np.sum,
    'air mean': np.mean, 
    'wind_speed min': min,
    'wind_speed max':max,
    'wind_speed sum':np.sum,
    'wind_speed mean': np.mean
})

In [205]:
statistics_per_month.head()

,uwnd min,uwnd mean,vwnd sum,vwnd mean,air min,air max,air sum,air mean,wind_speed min,wind_speed max,wind_speed sum,wind_speed mean
time,,,,,,,,,,,,
1985-01-31,-4.599991,1.825285,620.058685,0.555608,228.200012,264.799988,272532.838730,244.205053,0.072790,9.220193,3698.468643,3.314040
1985-02-28,-4.189987,1.125801,-797.862259,-0.791530,232.230011,263.179993,247808.597992,245.841863,0.036061,8.192881,2740.674001,2.718923
1985-03-31,-5.739990,2.529380,1381.638550,1.238027,234.530014,275.980011,287973.659180,258.040913,0.060013,11.423858,4777.891649,4.281265
1985-04-30,-8.349991,0.649248,77.458374,0.071721,250.280014,280.750000,292206.289383,270.561379,0.101969,11.615078,4105.048187,3.800971
1985-05-31,-8.119995,0.713198,244.128647,0.218753,269.080017,289.429993,311876.229614,279.458987,0.106298,9.824836,4147.772249,3.716642


In [204]:
statistics_per_month.to_pickle(os.path.join(files, 'statistics_per_month.pkl'))

In [208]:
statistics_per_quarter = statistics_per_day.resample('Q').agg({
    'uwnd min':min, 
    'uwnd min': min,
    'uwnd mean': np.mean, # mean of means :)
    'vwnd sum':  np.sum, 
    'vwnd sum':  np.sum, 
    'vwnd mean': np.mean, 
    'air min': min,
    'air max': max,
    'air sum': np.sum,
    'air mean': np.mean, 
    'wind_speed min': min,
    'wind_speed max':max,
    'wind_speed sum':np.sum,
    'wind_speed mean': np.mean
})

In [209]:
statistics_per_quarter.head()

,uwnd min,uwnd mean,vwnd sum,vwnd mean,air min,air max,air sum,air mean,wind_speed min,wind_speed max,wind_speed sum,wind_speed mean
time,,,,,,,,,,,,
1985-03-31,-5.739990,1.850190,1203.834976,0.371554,228.200012,275.980011,808315.095901,249.479968,0.036061,11.423858,11217.034294,3.462048
1985-06-30,-8.349991,0.459934,190.825134,0.058249,250.280014,299.900024,917290.728043,280.003275,0.101969,11.615078,11713.329626,3.575497
1985-09-30,-7.589996,0.245473,-807.134171,-0.243700,269.600006,301.309998,952705.096558,287.652505,0.042446,9.259055,10523.316355,3.177330
1985-12-31,-6.889984,2.425421,1283.765350,0.387610,229.600006,281.380005,852410.927643,257.370449,0.042446,11.040541,12507.263447,3.776348
1986-03-31,-6.019989,2.602980,3071.084381,0.947866,228.380005,273.400024,820379.808990,253.203645,0.120832,11.157224,13721.294561,4.234967


In [210]:
statistics_per_quarter.to_pickle(os.path.join(files, 'statistics_per_quarter.pkl'))